In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!wget "https://raw.githubusercontent.com/rslim087a/road-image/6b56f889e20871e1836b0045222d07dcea9ff8d6/test_image.jpg"
!wget "https://raw.githubusercontent.com/rslim087a/road-video/master/test2.mp4"

In [ ]:
def make_coord(img,line_param):
    slope,inter=line_param
    y1=img.shape[0]
    y2=int(y1*(1/2))
    x1=int((y1-inter)/slope)
    x2=int((y2-inter)/slope)
    return np.array([x1,y1,x2,y2])

In [ ]:
def avg_slope_intercept(img,lines):

    l_fit=[]
    r_fit=[]

    for line in lines:

        x1,y1,x2,y2=line.reshape(4)
        param=np.polyfit((x1,x2),(y1,y2),1)
        slope=param[0]
        inter=param[1]

        if slope<0:
            l_fit.append((slope,inter))
        else:
            r_fit.append((slope,inter))
        
    l_fit_avg=np.average(l_fit,axis=0)
    r_fit_avg=np.average(r_fit,axis=0)

    l_line=make_coord(img, l_fit_avg)
    r_line=make_coord(img, r_fit_avg)

    return np.array([l_line,r_line])

In [ ]:
def edge_detect(img):
    
    gray=cv2.cvtColor(ln_img,cv2.COLOR_RGB2GRAY)
    #blur=cv2.GaussianBlur(gray,(5,5),0)
    canny=cv2.Canny(gray,50,150)
    return canny

In [ ]:
def reg_of_int(img):
    
    ht=img.shape[0]
    pg=np.array([[(200,ht),(1100,ht),(550,250)]])
    mask=np.zeros_like(img)
    cv2.fillPoly(mask,pg,255)
    mk_img=cv2.bitwise_and(img,mask)
    return mk_img

In [ ]:
def disp_lines(img,lines):
    line_img=np.zeros_like(img)

    if lines is not None:

        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            cv2.line(line_img,(x1,y1),(x2,y2),(255,0,0),10)
    
    return line_img      

In [ ]:
def det_lanes(image):
    canny=edge_detect(image)
    cp_img=reg_of_int(canny)
    lines=cv2.HoughLinesP(cp_img,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    avg_lines=avg_slope_intercept(image,lines)
    line_img=disp_lines(image,avg_lines)

    cmb_img=cv2.addWeighted(image,0.8,line_img,1,1)

    cv2_imshow(cmb_img)

In [ ]:
def det_image():
    img=cv2.imread('test_image.jpg')
    ln_img=np.copy(img)

    det_lanes(ln_img)

In [ ]:
def det_video():
    cap=cv2.VideoCapture('test2.mp4')

    while(cap.isOpened()):

        _, frame=cap.read()

        det_lanes(frame)

        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()